In [1]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

## Task 1
Choose a reddit page you want to crawl

In [2]:
URL = "https://api.pushshift.io/reddit/submission/search/"  #query submissions
PARAMS = {
    'after': 1593561600, #get dates after 07/01/2020 @ 12:00am (UTC)
    'before': 1596240000, #get dates before 08/01/2020 @ 12:00am (UTC)
    'sort_type': 'score', # sort by score
    'sort': 'desc', # sort in descending order
    'subreddit': 'animalcrossing', # do a search on Animal Crossing subreddit
    'size': 20, # give only 20 search results
#     'fields': ["id","title","selftext","score","num_comments","created_utc"] #return only the following fields
}

#use the requests library to query pushshift api
r = requests.get(url = URL, params=PARAMS)
#parse returned data to a json object
r.json()

{'data': [{'all_awardings': [{'award_sub_type': 'GLOBAL',
     'award_type': 'global',
     'coin_price': 70,
     'coin_reward': 0,
     'count': 1,
     'days_of_drip_extension': 0,
     'days_of_premium': 0,
     'description': '*Lowers face into palm*',
     'end_date': None,
     'giver_coin_reward': 0,
     'icon_format': 'PNG',
     'icon_height': 2048,
     'icon_url': 'https://i.redd.it/award_images/t5_22cerq/ey2iodron2s41_Facepalm.png',
     'icon_width': 2048,
     'id': 'award_b1b44fa1-8179-4d84-a9ed-f25bb81f1c5f',
     'is_enabled': True,
     'is_new': False,
     'name': 'Facepalm',
     'penny_donate': 0,
     'penny_price': 0,
     'resized_icons': [{'height': 16,
       'url': 'https://preview.redd.it/award_images/t5_22cerq/ey2iodron2s41_Facepalm.png?width=16&amp;height=16&amp;auto=webp&amp;s=d06b7de23ce8b8ea0f3e7cfd15033ac4893b72f0',
       'width': 16},
      {'height': 32,
       'url': 'https://preview.redd.it/award_images/t5_22cerq/ey2iodron2s41_Facepalm.png?widt

## Task 2
The following fields should be present when you crawl (10 points):

- author
- subreddit
- date created 
- number of comments 
- score
- submission title 
- submission description

In [3]:
def to_utc(date):
    #This function converts an object to UTC. This is to automate the conversion 
    #of dates instead of going to https://www.unixtimeconverter.io/ 
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-07-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-08-01", "%Y-%m-%d")

#Create a range of dates to iterate 
date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())

#prepare the parameters needed to call the API
sort_type="score"
sort="desc"
fields=["author","subreddit","created_utc","num_comments","score", "title", "selftext", "id"]
subreddit = 'animalcrossing'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []
#loop through the dates 
for i, s_date in enumerate(date_range):
    #prevents us from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })

        #add logs 
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        #so that we dont get blocked from abusing the API we call it after pausing for 1 second
        time.sleep(1)

Doing 2020-07-01 to 2020-07-02
=====Done
Doing 2020-07-02 to 2020-07-03
=====Done
Doing 2020-07-03 to 2020-07-04
=====Done
Doing 2020-07-04 to 2020-07-05
=====Done
Doing 2020-07-05 to 2020-07-06
=====Done
Doing 2020-07-06 to 2020-07-07
=====Done
Doing 2020-07-07 to 2020-07-08
=====Done
Doing 2020-07-08 to 2020-07-09
=====Done
Doing 2020-07-09 to 2020-07-10
=====Done
Doing 2020-07-10 to 2020-07-11
=====Done
Doing 2020-07-11 to 2020-07-12
=====Done
Doing 2020-07-12 to 2020-07-13
=====Done
Doing 2020-07-13 to 2020-07-14
=====Done
Doing 2020-07-14 to 2020-07-15
=====Done
Doing 2020-07-15 to 2020-07-16
=====Done
Doing 2020-07-16 to 2020-07-17
=====Done
Doing 2020-07-17 to 2020-07-18
=====Done
Doing 2020-07-18 to 2020-07-19
=====Done
Doing 2020-07-19 to 2020-07-20
=====Done
Doing 2020-07-20 to 2020-07-21
=====Done
Doing 2020-07-21 to 2020-07-22
=====Done
Doing 2020-07-22 to 2020-07-23
=====Done
Doing 2020-07-23 to 2020-07-24
=====Done
Doing 2020-07-24 to 2020-07-25
=====Done
Doing 2020-07-25

In [5]:
results

[[{'author': 'Icairy',
   'created_utc': 1593613135,
   'id': 'hjb2c4',
   'num_comments': 13,
   'score': 22,
   'selftext': '',
   'subreddit': 'AnimalCrossing',
   'title': 'Bones, please never change'},
  {'author': 'alleesaurus',
   'created_utc': 1593613045,
   'id': 'hjb1bb',
   'num_comments': 47,
   'score': 22,
   'selftext': '',
   'subreddit': 'AnimalCrossing',
   'title': 'The rainy days in Animal Crossing are my favorites, so I made some art inspired by it (feat Freya and Lolly)!'},
  {'author': 'SketchFarm',
   'created_utc': 1593613889,
   'id': 'hjbb0s',
   'num_comments': 34,
   'score': 17,
   'selftext': '',
   'subreddit': 'AnimalCrossing',
   'title': 'Had to draw this room, I love sitting in the Butterfly Garden'},
  {'author': 'Sir_Benjamus',
   'created_utc': 1593612323,
   'id': 'hjatfq',
   'num_comments': 93,
   'score': 17,
   'selftext': '',
   'subreddit': 'AnimalCrossing',
   'title': "Isabelle didn't think it was worth mentioning, but Happy Canada Day!"

## Task 3
After crawling, save your results to a pandas dataframe (3 points).

In [35]:
flat_list = []

for sublist in results:
    if sublist is not None:
        for item in sublist:
            flat_list.append(item)

df = pd.DataFrame.from_dict(flat_list)
display(df.head())
df.to_csv("reddit_animalcrossing.csv")

,author,created_utc,id,num_comments,score,selftext,subreddit,title
0,Icairy,1593613135,hjb2c4,13,22,,AnimalCrossing,"Bones, please never change"
1,alleesaurus,1593613045,hjb1bb,47,22,,AnimalCrossing,The rainy days in Animal Crossing are my favor...
2,SketchFarm,1593613889,hjbb0s,34,17,,AnimalCrossing,"Had to draw this room, I love sitting in the B..."
3,Sir_Benjamus,1593612323,hjatfq,93,17,,AnimalCrossing,"Isabelle didn't think it was worth mentioning,..."
4,Gahrenn,1593616166,hjc1rq,117,16,,AnimalCrossing,I just wanna buy some art.


In [7]:
df.to_csv(r'/Users/Janine/Desktop/DLSU/Module_1_Data_Collection/reddit_animalcrossing.csv')

In [36]:
df.describe()

,created_utc,num_comments,score
count,3.200000e+03,3200.000000,3200.000000
mean,1.594947e+09,31.864062,598.858750
std,7.940632e+05,88.451561,2790.592697
min,1.593612e+09,0.000000,1.000000
25%,1.594288e+09,2.000000,2.000000
50%,1.594944e+09,8.000000,15.000000
75%,1.595635e+09,28.000000,174.000000
max,1.596323e+09,1538.000000,47568.000000


## Task 4
Answer the following questions (12 points):
- How many submissions were you able to gather? 
- Who has the most submissions? 
- Which submission has the highest score? 
- Which submission has the highest number of comments?
- Which day of the week has the most submissions? 

Tip:** _For item#4, recall how to use the aggregation functions in pandas like count, value_counts, sum, etc. For getting the day of the week, look into how to get the dayofweek from a datetime object in pandas. (Hint: You may need to use pd.to_datetime to convert your date column...)_

In [37]:
# How many submissions were you able to gather?

len(df)

3200

In [38]:
#Who has the most submissions?

df['author'].value_counts(sort=True).head(1)

SwixSwax    21
Name: author, dtype: int64

In [39]:
# Which submission has the highest score?
# code source: https://stackoverflow.com/questions/15741759/find-maximum-value-of-a-column-and-return-the-corresponding-row-values-using-pan

df[df['score']==df['score'].max()]

,author,created_utc,id,num_comments,score,selftext,subreddit,title
1700,Spidey0062,1595092370,htk602,1538,47568,,AnimalCrossing,Accurate


In [41]:
# Which submission has the highest number of comments?
# code source: https://stackoverflow.com/questions/15741759/find-maximum-value-of-a-column-and-return-the-corresponding-row-values-using-pan

df[df['num_comments']==df['num_comments'].max()]

,author,created_utc,id,num_comments,score,selftext,subreddit,title
1700,Spidey0062,1595092370,htk602,1538,47568,,AnimalCrossing,Accurate


In [34]:
# code guide: Epoch Timestamps: https://pandas-docs.github.io/pandas-docs-travis/user_guide/timeseries.html
# code guide: Day of Week: https://stackoverflow.com/questions/9847213/how-do-i-get-the-day-of-week-given-a-date

df['timeStamp'] = pd.to_datetime(df['created_utc'], unit='s')
print(df['timeStamp'])

0      2020-07-01 14:18:55
1      2020-07-01 14:17:25
2      2020-07-01 14:31:29
3      2020-07-01 14:05:23
4      2020-07-01 15:09:26
               ...        
3195   2020-08-01 18:37:06
3196   2020-08-01 07:56:35
3197   2020-08-01 19:19:54
3198   2020-08-01 19:16:24
3199   2020-08-01 18:41:21
Name: timeStamp, Length: 3200, dtype: datetime64[ns]


In [32]:
df['Day of Week'] = df['timeStamp'].apply(lambda time: time.dayofweek)
df['Day of Week'].value_counts(sort=True)

5    500
3    500
4    500
2    500
1    400
6    400
0    400
Name: Day of Week, dtype: int64

In [33]:
# Which day of the week has the most submissions?
# code guide: Day int to Calendar day: https://stackoverflow.com/questions/36341484/get-day-name-from-weekday-int

import calendar
calendar.day_name[5]

'Saturday'